In [1]:
from translation_utils import translate_column
import pandas as pd
from datetime import datetime

In [2]:
# Import allGoogleReviews_2025-04-05
allGoogleReviews_2025_04_05 = pd.read_excel("reviews/google/allGoogleReviews_2025-04-05.xlsx")

# Filter out where originalLanguage is "en"
filtered_df = allGoogleReviews_2025_04_05[allGoogleReviews_2025_04_05['originalLanguage'] != "en"]

# Filter out where responseFromOwnerText is shorter than 5 characters
filtered_df = filtered_df[filtered_df['responseFromOwnerText'].str.len() > 5]

# Filter out where responseFromOwnerText_en is not null
filtered_df = filtered_df[filtered_df['responseFromOwnerText_en'].isnull()]

# Count the total number of characters in responseFromOwnerText
total_characters = filtered_df['responseFromOwnerText'].str.len().sum()
print(f"Total characters in responseFromOwnerText: {total_characters}")


Total characters in responseFromOwnerText: 711884


In [3]:
# If you want to limit the total characters to 450000, you need to select rows until you reach that limit
if total_characters > 450000:
    # Create a cumulative sum of character lengths
    filtered_df['cumulative_chars'] = filtered_df['responseFromOwnerText'].str.len().cumsum()
    # Filter to keep only rows where cumulative characters are less than 450000
    filtered_df = filtered_df[filtered_df['cumulative_chars'] <= 450000]
    # Remove the temporary column
    filtered_df = filtered_df.drop('cumulative_chars', axis=1)

# First count the total characters
total_characters = filtered_df['responseFromOwnerText'].str.len().sum()
print(f"Total characters in responseFromOwnerText: {total_characters}")


Total characters in responseFromOwnerText: 449547


In [4]:
# Find the difference between allGoogleReviews_2025-04-05 and filtered_df based on reviewId and save them to a new pd dataframe
difference_df = allGoogleReviews_2025_04_05[~allGoogleReviews_2025_04_05['reviewId'].isin(filtered_df['reviewId'])]

In [5]:
# Get the translated DataFrame without saving
translated_df, chars_translated = translate_column(
    input_data=filtered_df,  # or file path
    source_column="responseFromOwnerText",
    target_column="responseFromOwnerText_en",  # optional
    translation_service="deepl",  # optional
    output_path=None  # Don't save, just return the DataFrame
)

# Do additional processing on translated_df
# Combine the translated_df with the difference_df based on reviewId
combined_df = pd.concat([difference_df, translated_df])

# Save the processed DataFrame when you're ready
output_path = "C:/Users/yigit/Desktop/Enterprises/verisanus_2/a_dev/reviews/google/processed_reviews.xlsx"
combined_df.to_excel(output_path, index=False)

KeyError: 'api_settings'